In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import cvxpy as cp
from typing import Dict, Any, Tuple

## Production System Class
This class randomly generates a fleet of convex generators. Parameters: `n_machines` controls fleet size, `seed` ensures reproducibility, `alpha/beta/gamma_bounds` define the cost coefficients, and `capacity_bounds` set min/max outputs. Methods `describe()` and `total_capacity()` summarize the fleet.

In [2]:
class ProductionSystem:
    """
    Holds the convex generator fleet (cost curves + capacity bounds).
    """
    def __init__(self, n_machines: int = 5, seed: int = 42,
                 alpha_bounds: Tuple[float, float] = (0.05, 0.2),
                 beta_bounds: Tuple[float, float] = (2.0, 5.0),
                 gamma_bounds: Tuple[float, float] = (10.0, 20.0),
                 capacity_bounds: Tuple[float, float] = (50.0, 150.0)):
        rng = np.random.default_rng(seed)
        self.n = n_machines
        
        # Quadratic coefficients cost(x) = alpha x^2 + beta x + gamma
        self.alpha = rng.uniform(alpha_bounds[0], alpha_bounds[1], n_machines)
        self.beta = rng.uniform(beta_bounds[0], beta_bounds[1], n_machines)
        self.gamma = rng.uniform(gamma_bounds[0], gamma_bounds[1], n_machines)
        
        # Lower/upper capacity bounds (MW)
        self.l = np.zeros(n_machines)
        self.u = rng.uniform(capacity_bounds[0], capacity_bounds[1], n_machines)
        
    def describe(self) -> pd.DataFrame:
        machines = np.arange(1, self.n + 1)
        data = {
            "alpha": self.alpha,
            "beta": self.beta,
            "gamma": self.gamma,
            "min_cap": self.l,
            "max_cap": self.u,
        }
        return pd.DataFrame(data, index=machines).rename_axis("machine")
        
    def total_capacity(self) -> float:
        return float(np.sum(self.u))

## Economic Dispatch Solver
`solve_dispatch` enforces chance constraints. Inputs: `system` (fleet), `mu_D`/`sigma_D` (demand stats), `reliability` (probability level), and `mode` (Gaussian vs. Chebyshev). It builds CVXPY variable `x`, computes effective demand `D_eff`, sets up the quadratic cost objective, enforces capacity/demand constraints, solves, and returns primal/dual results.

In [ ]:
def solve_dispatch(
    system: ProductionSystem,
    mu_D: float,
    sigma_D: float,
    reliability: float = 0.95,
    mode: str = "normal",
) -> Dict[str, Any]:
    """Solve the stochastic economic dispatch under Gaussian or Chebyshev risk."""
    
    if sigma_D < 0 or mu_D < 0:
        raise ValueError("Demand statistics must be non-negative")
    if not 0.0 < reliability < 1.0:
        raise ValueError("Reliability must lie strictly between 0 and 1")
    
    x = cp.Variable(system.n)
    alpha_risk = 1.0 - reliability
    
    if mode == "normal":
        z_score = stats.norm.ppf(reliability)
        D_eff = mu_D + z_score * sigma_D
    elif mode == "robust":
        k_robust = np.sqrt((1 - alpha_risk) / alpha_risk)
        D_eff = mu_D + k_robust * sigma_D
    else:
        raise ValueError("Mode must be 'normal' or 'robust'")
    
    if D_eff > system.total_capacity() + 1e-6:
        raise ValueError("Effective demand exceeds installed capacity")
    
    objective = cp.Minimize(
        cp.sum(cp.multiply(system.alpha, x**2) + cp.multiply(system.beta, x))
    )
    
    c_demand = [cp.sum(x) >= D_eff]
    c_upper = [x <= system.u]
    c_lower = [x >= system.l]
    constraints = c_demand + c_upper + c_lower
    
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.OSQP, warm_start=True)
    
    if prob.status not in {"optimal", "optimal_inaccurate"}:
        return {"status": prob.status}
    
    dispatch = x.value
    total_cost = float(
        np.sum(system.alpha * dispatch**2 + system.beta * dispatch + system.gamma)
    )
    
    return {
        "status": prob.status,
        "x": dispatch,
        "cost": total_cost,
        "objective": prob.value,
        "D_eff": float(D_eff),
        "lambda": c_demand[0].dual_value,
        "nu_u": c_upper[0].dual_value,
        "nu_l": c_lower[0].dual_value,
    }